<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/MultivariateWienerPath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Univariate Wiener Path Construction

In [219]:
import torch
import time

import math
import numpy as np

In [220]:
SQRT_2 = np.sqrt(2)

In [221]:
class UnivariateBrownianBridge():
  def __init__(self, number_time_steps):
    self.number_time_steps = number_time_steps

    self.left_index = torch.zeros(number_time_steps, dtype=int)
    self.right_index = torch.zeros(number_time_steps, dtype=int)
    self.bridge_index = torch.zeros(number_time_steps, dtype=int)
    self.left_weight = torch.zeros(number_time_steps)
    self.right_weight = torch.zeros(number_time_steps)
    self.std_dev = torch.zeros(number_time_steps)

    self._map = torch.zeros(number_time_steps, dtype=int)

    self._map[-1] = 1
    self.bridge_index[0] = number_time_steps - 1
    self.std_dev[0] = torch.sqrt(torch.tensor(1.0) * number_time_steps)
    self.left_weight[0] = 0
    self.right_weight[0] = 0

    j=0
    for i in range(1,number_time_steps):
      while self._map[j] == True:
        j = j + 1
      k = j
      while self._map[k] == False:
        k = k + 1
      l = j+((k-1-j)>>1)
      self._map[l]=i
      self.bridge_index[i]=l
      self.left_index[i]=j
      self.right_index[i]=k
      self.left_weight[i]=(k-l)/(k+1-j)
      self.right_weight[i]=(1+l-j)/(k+1-j)
      self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
      j=k+1
      if j>=number_time_steps:
        j=0
        
  @torch.jit.script
  def buildPath(path, z, number_time_steps: int, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
    path[-1] = std_dev[0]*z[0]
    j = 0
    k = 0
    l = 0
    i = 0
    for i in range(1,number_time_steps):
      j = left_index[i]
      k = right_index[i]
      l = bridge_index[i]
      lw = left_weight[i]
      rw = right_weight[i]
      sd = std_dev[i]
      if j > 0:
        path[l] = path[j-1] * lw + path[k] * rw + z[i] * sd
      else:
        path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

  @torch.jit.script
  def buildPathIncrements(path, z, number_time_steps: int, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
    path[-1] = std_dev[0]*z[0]
    j = 0
    k = 0
    l = 0
    i = 0
    for i in range(1,number_time_steps):
      j = left_index[i]
      k = right_index[i]
      l = bridge_index[i]
      lw = left_weight[i]
      rw = right_weight[i]
      sd = std_dev[i]
      if j > 0:
        path[l] = path[j-1] * lw + path[k] * rw + z[i] * sd
      else:
        path[l] = right_weight[i] * path[k] + std_dev[i] * z[i]

    for i in range(1, number_time_steps):
      path[-i] = path[-i] - path[-(i+1)]
    

  def path(self, path, z):
    return self.buildPath(path, z, self.number_time_steps, self.left_index, self.right_index, self.bridge_index, self.left_weight, self.right_weight, self.std_dev)

  def pathIncrements(self, path, z):
    return self.buildPathIncrements(path, z, self.number_time_steps, self.left_index, self.right_index, self.bridge_index, self.left_weight, self.right_weight, self.std_dev)    

In [222]:
brownian = UnivariateBrownianBridge(8)

In [223]:
print(UnivariateBrownianBridge.buildPath.code)

def buildPath(path: Tensor,
    z: Tensor,
    number_time_steps: int,
    left_index: Tensor,
    right_index: Tensor,
    bridge_index: Tensor,
    left_weight: Tensor,
    right_weight: Tensor,
    std_dev: Tensor) -> None:
  _0 = torch.mul(torch.select(std_dev, 0, 0), torch.select(z, 0, 0))
  _1 = torch.copy_(torch.select(path, 0, -1), _0, False)
  _2 = torch.__range_length(1, number_time_steps, 1)
  for _3 in range(_2):
    i = torch.__derive_index(_3, 1, 1)
    j = torch.select(left_index, 0, i)
    _4 = annotate(int, j)
    k = torch.select(right_index, 0, i)
    _5 = annotate(int, k)
    l = torch.select(bridge_index, 0, i)
    _6 = annotate(int, l)
    lw = torch.select(left_weight, 0, i)
    rw = torch.select(right_weight, 0, i)
    sd = torch.select(std_dev, 0, i)
    if torch.gt(_4, 0):
      _7 = torch.select(path, 0, torch.sub(_4, 1))
      _8 = torch.mul(_7, lw)
      _9 = torch.mul(torch.select(path, 0, _5), rw)
      _10 = torch.add(_8, _9, alpha=1)
      _11 = torch.m

In [224]:
sobol_engine =  torch.quasirandom.SobolEngine(8)

x = sobol_engine.draw(2)

print(x)

y = torch.transpose(torch.erfinv(2.*(x-0.5))*SQRT_2,0,1)

print(y)

#x = torch.zeros(size=(8,2))
#x[0] = 1

path = torch.zeros(size=(8,2))

#brownian.path(path, x)
brownian.pathIncrements(path, y)

path

tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
        [0.7500, 0.2500, 0.7500, 0.2500, 0.7500, 0.2500, 0.7500, 0.2500]])
tensor([[ 0.0000,  0.6745],
        [ 0.0000, -0.6745],
        [ 0.0000,  0.6745],
        [ 0.0000, -0.6745],
        [ 0.0000,  0.6745],
        [ 0.0000, -0.6745],
        [ 0.0000,  0.6745],
        [ 0.0000, -0.6745]])


tensor([[ 0.0000,  0.8142],
        [ 0.0000, -0.1397],
        [ 0.0000, -0.8142],
        [ 0.0000,  0.1397],
        [ 0.0000,  0.6166],
        [ 0.0000, -0.3372],
        [ 0.0000,  0.3372],
        [ 0.0000,  1.2911]], grad_fn=<CopySlices>)

In [225]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
numberTimeSteps =  128#@param {type:"integer"}
numberSimulation =  200000#@param {type:"integer"}


# First run (includes graph optimization time)
time_start = time.time()
path = torch.zeros(size=(numberTimeSteps,numberSimulation))

brownian = UnivariateBrownianBridge(numberTimeSteps)

sobol_engine =  torch.quasirandom.SobolEngine(numberTimeSteps)

x = sobol_engine.draw(numberSimulation)
y = torch.transpose(torch.erfinv(2.*(x-0.5))*SQRT_2,0,1)
brownian.pathIncrements(path, y)
time_end = time.time()
time_price_cpu = time_end - time_start
print("First time on a CPU: ", time_price_cpu)

First time on a CPU:  0.3289988040924072


In [226]:
import torch
import torchvision.models as models
import torch.autograd.profiler as profiler

In [229]:
numberTimeSteps =  128#@param {type:"integer"}
numberSimulation =  200000#@param {type:"integer"}

brownian = UnivariateBrownianBridge(numberTimeSteps)

path = torch.zeros(size=(numberTimeSteps,numberSimulation))

with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("univariate_bridge"):
        x = sobol_engine.draw(numberSimulation)
        y = torch.transpose(torch.erfinv(2.*(x-0.5))*SQRT_2,0,1)
        brownian.pathIncrements(path, y)


In [230]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

------------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name                                  Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     Number of Calls  
------------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
univariate_bridge                     19.70%           62.154ms         100.00%          315.473ms        315.473ms        1                
mul                                   38.78%           122.347ms        38.78%           122.347ms        324.529us        377              
FusionGroup                           0.69%            2.183ms          33.03%           104.195ms        820.433us        127              
_sobol_engine_draw                    22.38%           70.592ms         24.46%           77.155ms         77.155ms         1                
sub         

# MultivariateBrownian

In [285]:
numberSimulation = 1
numberTimeSteps = 3
numberStates = 2
dim = numberTimeSteps * numberStates

In [286]:
sobol_engine =  torch.quasirandom.SobolEngine(dim)
x = sobol_engine.draw(numberSimulation)
x = torch.transpose(x,0,1)

In [287]:
y = torch.reshape(x,shape=(numberTimeSteps,numberStates,numberSimulation))

In [288]:
m = torch.zeros(size=(numberStates,numberStates))
m[0,0] = 1
m[1,0] = 1
m[1,1] = 0

In [289]:
w = torch.matmul(m,y[0,:,:])

In [290]:
w

tensor([[0.5000],
        [0.5000]])

In [291]:
def square_root_symmetric_matrix(A):
    w, v = torch.symeig(A, eigenvectors=True)
    return torch.mm(torch.mm(v, torch.diag(torch.sqrt(w[:]))), v.t())


In [292]:
class MultivariateBrownianBridge():
    def __init__(self, forwardCovarianceMatrices):
        self.forwardCovarianceMatrices = forwardCovarianceMatrices
        self.numberTimeSteps = len(forwardCovarianceMatrices)
        self.numberStates = len(forwardCovarianceMatrices[0])
        self.brownian = UnivariateBrownianBridge(numberTimeSteps)
        self.sqrtForwardCovarianceMatrices = torch.zeros(size=(self.numberTimeSteps, self.numberStates, self.numberStates))
        for i in range(self.numberTimeSteps):
            self.sqrtForwardCovarianceMatrices[i] = square_root_symmetric_matrix(self.forwardCovarianceMatrices[i])

    def pathIncrements(self, path, z):
        self.brownian.pathIncrements(path, z)
        for i in range(numberTimeSteps):
            path[i] = torch.matmul(self.sqrtForwardCovarianceMatrices[i], path[i])

In [293]:
path = torch.zeros(size=(numberTimeSteps, numberStates, numberSimulation))

In [294]:
fwd_cov = torch.ones(size=(numberTimeSteps, numberStates, numberStates))

In [295]:
multivariate_brownian = MultivariateBrownianBridge(fwd_cov)

In [296]:
multivariate_brownian.pathIncrements(path, y)

In [297]:
path

tensor([[[ 0.9856],
         [ 0.9856]],

        [[ 0.6196],
         [ 0.6196]],

        [[-0.3804],
         [-0.3804]]], grad_fn=<CopySlices>)

In [318]:
numberTimeSteps = 1
numberStates = 16
dim = numberTimeSteps * numberStates
numberSimulation =  200000#@param {type:"integer"}

fwd_cov = torch.ones(size=(numberTimeSteps, numberStates, numberStates))

multivariate_brownian = MultivariateBrownianBridge(fwd_cov)

path = torch.zeros(size=(numberTimeSteps, numberStates, numberSimulation))

with profiler.profile(record_shapes=True) as prof:
    with profiler.record_function("univariate_bridge"):
        sobol_engine =  torch.quasirandom.SobolEngine(dim)
        x = sobol_engine.draw(numberSimulation)
        x = torch.transpose(x,0,1)
        y = torch.reshape(x,shape=(numberTimeSteps,numberStates,numberSimulation))
        z = torch.erfinv(2.*(y-0.5))*SQRT_2
        multivariate_brownian.pathIncrements(path, z)

In [319]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

------------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
Name                                  Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     Number of Calls  
------------------------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  
univariate_bridge                     43.22%           27.021ms         99.99%           62.513ms         62.513ms         1                
erfinv                                27.59%           17.248ms         27.59%           17.248ms         17.248ms         1                
_sobol_engine_draw                    12.19%           7.618ms          13.58%           8.490ms          8.490ms          1                
mul                                   7.51%            4.695ms          7.51%            4.695ms          1.565ms          3                
matmul      